# Operacoes basicas com Spark DataFrames - Parte 2

### Trabalhando com missing datas

In [0]:
import numpy as np
import pandas as pd

In [0]:
X = np.c_[[np.nan, 1,2,3,4,5], [np.nan, 10, 20, np.nan, 38, 40], [5, 10, 15, 20, 30, 1]]
df_pandas = pd.DataFrame(X, columns=['colA', 'colB', 'colC'])
df_pandas

,colA,colB,colC
0,NaN,NaN,5.0
1,1.0,10.0,10.0
2,2.0,20.0,15.0
3,3.0,NaN,20.0
4,4.0,38.0,30.0
5,5.0,40.0,1.0


In [0]:
df = spark.createDataFrame(df_pandas)
df.show()

+----+----+----+
|colA|colB|colC|
+----+----+----+
|null|null| 5.0|
| 1.0|10.0|10.0|
| 2.0|20.0|15.0|
| 3.0|null|20.0|
| 4.0|38.0|30.0|
| 5.0|40.0| 1.0|
+----+----+----+



In [0]:
# Deletando todos os valores na
df.na.drop().show()

# Deletando apenas de uma coluna especifica
df.na.drop(subset='colA').show()

# Fazendo o preenchimento de todos os valores null
df.na.fill(100).show()

# Fazendo o preenchimento de valores especificos de uma coluna
df.na.fill(6, subset="colA").show()

+----+----+----+
|colA|colB|colC|
+----+----+----+
| 1.0|10.0|10.0|
| 2.0|20.0|15.0|
| 4.0|38.0|30.0|
| 5.0|40.0| 1.0|
+----+----+----+

+----+----+----+
|colA|colB|colC|
+----+----+----+
| 1.0|10.0|10.0|
| 2.0|20.0|15.0|
| 3.0|null|20.0|
| 4.0|38.0|30.0|
| 5.0|40.0| 1.0|
+----+----+----+

+-----+-----+----+
| colA| colB|colC|
+-----+-----+----+
|100.0|100.0| 5.0|
|  1.0| 10.0|10.0|
|  2.0| 20.0|15.0|
|  3.0|100.0|20.0|
|  4.0| 38.0|30.0|
|  5.0| 40.0| 1.0|
+-----+-----+----+

+----+----+----+
|colA|colB|colC|
+----+----+----+
| 6.0|null| 5.0|
| 1.0|10.0|10.0|
| 2.0|20.0|15.0|
| 3.0|null|20.0|
| 4.0|38.0|30.0|
| 5.0|40.0| 1.0|
+----+----+----+



### Joins com PySpark
* Vamos comecar criando 2 tabelas para trabalharmos o conceito de joins:

In [0]:
ID1 = np.random.choice(np.arange(20), size=15, replace=False)
ID2 = np.random.choice(np.arange(20), size=15, replace=False)
age = np.random.randint(20,70, size=15)
sexo = np.random.choice(['M','F'],size=15)
profissao = np.random.choice(['analista', 'cientista', 'engenheiro', 'desenvolvedor'],size=15)

df1 = spark.createDataFrame(pd.DataFrame(np.c_[ID1, age, sexo], columns = ['ID', 'idade', 'sexo']))
df2 = spark.createDataFrame(pd.DataFrame(np.c_[ID2, profissao], columns = ['ID', 'profissao']))

In [0]:
# Inner join
df1.join(df2, on='ID', how='inner').show()
df1.join(df2, on='ID', how='outer').show()
df1.join(df2, on='ID', how='left').show()

+---+-----+----+-------------+
| ID|idade|sexo|    profissao|
+---+-----+----+-------------+
| 10|   32|   M|     analista|
| 11|   27|   M|desenvolvedor|
| 12|   60|   M|desenvolvedor|
| 13|   26|   F|   engenheiro|
| 15|   49|   M|desenvolvedor|
| 16|   51|   F|    cientista|
| 18|   48|   F|desenvolvedor|
| 19|   44|   F|     analista|
|  2|   58|   F|    cientista|
|  3|   22|   F|   engenheiro|
|  5|   20|   M|     analista|
|  6|   64|   M|   engenheiro|
+---+-----+----+-------------+

+---+-----+----+-------------+
| ID|idade|sexo|    profissao|
+---+-----+----+-------------+
|  0|   46|   F|         null|
|  1| null|null|     analista|
| 10|   32|   M|     analista|
| 11|   27|   M|desenvolvedor|
| 12|   60|   M|desenvolvedor|
| 13|   26|   F|   engenheiro|
| 14| null|null|desenvolvedor|
| 15|   49|   M|desenvolvedor|
| 16|   51|   F|    cientista|
| 17|   43|   F|         null|
| 18|   48|   F|desenvolvedor|
| 19|   44|   F|     analista|
|  2|   58|   F|    cientista|
|  3|  

* Qual a idade media dos analistas do sexo masculino?

In [0]:
import pyspark.sql.functions as F

df1.join(df2, on='ID', how='outer').show()

+---+-----+----+-------------+
| ID|idade|sexo|    profissao|
+---+-----+----+-------------+
|  0|   46|   F|         null|
|  1| null|null|     analista|
| 10|   32|   M|     analista|
| 11|   27|   M|desenvolvedor|
| 12|   60|   M|desenvolvedor|
| 13|   26|   F|   engenheiro|
| 14| null|null|desenvolvedor|
| 15|   49|   M|desenvolvedor|
| 16|   51|   F|    cientista|
| 17|   43|   F|         null|
| 18|   48|   F|desenvolvedor|
| 19|   44|   F|     analista|
|  2|   58|   F|    cientista|
|  3|   22|   F|   engenheiro|
|  4|   24|   F|         null|
|  5|   20|   M|     analista|
|  6|   64|   M|   engenheiro|
|  8| null|null|desenvolvedor|
+---+-----+----+-------------+



In [0]:
df1.join(df2, on='ID', how='outer')\
    .groupBy('sexo', 'profissao')\
    .agg(F.mean('idade').alias('idade_media'))\
    .filter((F.col('sexo') == 'M') & (F.col('profissao') == 'analista')).show()

+----+---------+-----------+
|sexo|profissao|idade_media|
+----+---------+-----------+
|   M| analista|       26.0|
+----+---------+-----------+

